In [10]:
import sys
from pathlib import Path

import pandas as pd
import os 
import numpy as np

TOP_DIR = Path('../../../').resolve()

if str(TOP_DIR) not in sys.path: sys.path.append(str(TOP_DIR))

SITE_DATA_DIR = '../../../docs/metrics/partnerships/_data'

PARTNERSHIPS_DATA = '../../../docs/_data/metrics/partnerships/partnerships.csv'
COUNTRIES_DATA = '../../../docs/_data/metrics/partnerships/countries.csv'

In [11]:
os.makedirs(SITE_DATA_DIR, exist_ok=True)

partnerships_data = pd.read_csv(PARTNERSHIPS_DATA, parse_dates=['start_date'])
countries_data = pd.read_csv(COUNTRIES_DATA)

FileNotFoundError: [Errno 2] No such file or directory: '../../../docs/metrics/partnerships/partnerships.csv'

Summarise

In [3]:
pd.Series({
    'total_partnerships': len(partnerships_data),
    'total_international': partnerships_data.international_element_to_project_.count(),
    'total_unique_countries': len(countries_data.unique_countries),
    'total_events': partnerships_data.total_number_of_events.sum(),
    'total_audience': partnerships_data.audience_numbers.sum(),
    'total_digital_audience': partnerships_data.digital_audiences.sum(),
    'total_participants': partnerships_data.number_of_participants.sum(),
    'total_in_person_events': partnerships_data['online_in_person_both'].value_counts()['In person'],
    'both_online_in_person_events': partnerships_data['online_in_person_both'].value_counts()['Both'],
    'online_only_events': partnerships_data['online_in_person_both'].value_counts()['Online']
}).astype(int).to_json(
    os.path.join(SITE_DATA_DIR, 'headlines.json'),
    date_format='iso', indent=2
)

Summarise by ward

In [4]:
partnerships_data.fillna('UNKNOWN').groupby('partner_venue_ward_code').partner_organisation.count().to_csv(os.path.join(SITE_DATA_DIR, 'partnerships_by_ward.csv'))

Summarise by month

In [5]:
partnerships_data.groupby('start_date').partner_organisation.count().resample('MS').sum().cumsum().to_csv(os.path.join(SITE_DATA_DIR, 'partnerships_by_month.csv'))

Event type

In [6]:
event_type = pd.DataFrame({
    'event_type': partnerships_data.groupby('online_in_person_both').partner_organisation.count()
}).reset_index().to_csv(os.path.join(SITE_DATA_DIR, 'event_type.csv'), index=False)



Project Artform

In [7]:
artform = pd.DataFrame({
    'number_of_partners': partnerships_data.groupby('project_artform').partner_organisation.count()
}).T.rename(columns={
    'Combined arts': 'combined_arts',
    'Dance': 'dance',
    'Literature': 'literature',
    'Music': 'music',
    'Theatre': 'theatre',
    'Visual art': 'visual_art'
}).reset_index().to_csv(os.path.join(SITE_DATA_DIR, 'artforms.csv'), index=False)


Project outputs

In [8]:

# Clean up the Project Outputs column 

partnerships_data.project_output


0                                       ['Performance']
1                                       ['Performance']
2     ['Performance', 'Exhibition', 'Workshop', 'Oth...
3                           ['Performance', 'Workshop']
4                           ['Performance', 'Workshop']
                            ...                        
61                                        ['Residency']
62                                        ['Residency']
63    ['Performance', 'Pop-up', 'Public Artwork (scu...
64                                      ['Performance']
65                ['Performance', 'Workshop', 'Other ']
Name: project_output, Length: 66, dtype: object

In [9]:
outputs = pd.DataFrame({
    'number_of_partners': partnerships_data.groupby('project_output').partner_organisation.count()
}).reset_index()
